In [ ]:
from modelo.problema import Problema
from modelo.reporte import Reporte
import pandas as pd
from datetime import datetime

In [ ]:
file = '0_model_21sept-2_rev.xlsm'

problema = Problema(excel_file_path=file)

problema.generar_sets()

problema.generar_parameters()

problema.generar_vars()

problema.generar_target()

problema.gen_constrains()

problema.solve(tlimit=30)

In [ ]:
reporte = Reporte(problema=problema)

df_dict = reporte.df_dict

In [ ]:
# Fletes variables
fletes_variables_df = df_dict['flete_variable'].copy()
fletes_variables_df.drop(columns=['tipo'], inplace=True)
fletes_variables_df.rename(columns={'costo_por_kg':'costo_flete_por_kg'}, inplace=True)

# fletes fijos
fletes_fijos_df = df_dict['flete_fijo'].copy()
fletes_fijos_df.drop(columns=['tipo'], inplace=True)
fletes_fijos_df.rename(columns={'costo_por_camion':'costo_flete_por_camion'}, inplace=True)

# intercompany
intercompany_df = df_dict['costo_intercompany'].copy()
intercompany_df.drop(columns=['tipo'], inplace=True)

# Costo de almacenamiento en bodega de puerto
costo_almacenamiento_df = df_dict['costo_almacenamiento_puerto'].copy()
costo_almacenamiento_df.drop(columns=['tipo'], inplace=True)
costo_almacenamiento_df.rename(columns={'costo_por_kg':'costo_corte_por_kg'}, inplace=True)

# Valor CIF de la carga
valor_cif_df = df_dict['valor_cif'].copy()
valor_cif_df.drop(columns=['tipo'], inplace=True)

# Llegadas al puerto
llegadas_puerto_df = df_dict['Llegadas_a_puerto'].copy()
llegadas_puerto_df.drop(columns=['tipo'], inplace=True)
llegadas_puerto_df.rename(columns={'kg':'descarga_barco_kg'}, inplace=True)


# Costo de operación portuaria por despacho directo
costo_op_directo_df = df_dict['cost_oportuaria_directo'].copy()
costo_op_directo_df.drop(columns=['tipo'], inplace=True)
costo_op_directo_df.rename(columns={'costo_por_kg': 'costo_portuario_directo_kg'}, inplace=True)

# Costo de operación portuaria por despacho directo
costo_op_bodegaje_df = df_dict['costo_oportuaria_bodegaje'].copy()
costo_op_bodegaje_df.drop(columns=['tipo'], inplace=True)
costo_op_bodegaje_df.rename(columns={'costo_por_kg': 'costo_portuario_bodegaje_kg'}, inplace=True)


# Inventario inicial
inventario_inicial_puerto_df = df_dict['inv_inicial_cargas'].copy()
inventario_inicial_puerto_df.drop(columns=['tipo'], inplace=True)
inventario_inicial_puerto_df.rename(columns={'kg':'inventario_al_cierre_kg'}, inplace=True)
inventario_inicial_puerto_df['periodo'] = '-1'

# Inventario en puerto
inventario_puerto_df = df_dict['inventario_puerto'].copy()
inventario_puerto_df.drop(columns=['tipo'], inplace=True)
inventario_puerto_df.rename(columns={'kg':'inventario_al_cierre_kg'}, inplace=True)
print(inventario_puerto_df.shape)
inventario_puerto_df.head()

# Armar el fact del inventario en puerto
fact_inventario_puerto = pd.concat([inventario_inicial_puerto_df, inventario_puerto_df])


# Despachos directos
despacho_directo_df = df_dict['despacho_directo'].copy()
despacho_directo_df.drop(columns=['tipo'])
despacho_directo_df.rename(
    columns={'cantidad_camiones': 'cantidad_camiones_directo'}, inplace=True)
despacho_directo_df['kilos_despachados_directo'] = 34000 * \
    despacho_directo_df['cantidad_camiones_directo']

# Colocar costos de operación por depacho directo
despacho_directo_df = pd.merge(left=despacho_directo_df,
                               right=costo_op_directo_df,
                               left_on=['operador', 'ingrediente'],
                               right_on=['operador', 'ingrediente'],
                               how='left')

# Colocar fletes variables al despacho directo
despacho_directo_df = pd.merge(left=despacho_directo_df,
                               right=fletes_variables_df,
                               left_on=['operador', 'ingrediente',
                                        'empresa_destino', 'planta'],
                               right_on=['operador', 'ingrediente',
                                         'empresa', 'planta'],
                               how='left').drop(columns=['empresa'])

# Colocar fletes fijos al despacho directo
despacho_directo_df = pd.merge(left=despacho_directo_df,
                               right=fletes_fijos_df,
                               left_on=['operador', 'ingrediente',
                                        'empresa_destino', 'planta'],
                               right_on=['operador', 'ingrediente',
                                         'empresa', 'planta'],
                               how='left').drop(columns=['empresa'])

# Colocar valor Cif
despacho_directo_df = pd.merge(left=despacho_directo_df,
                               right=valor_cif_df,
                               left_on=['empresa_origen', 'operador',
                                        'importacion', 'ingrediente'],
                               right_on=['empresa', 'operador',
                                         'importacion', 'ingrediente'],
                               how='left').drop(columns=['empresa'])


# Colocar costo intercompany al despacho directo
despacho_directo_df = pd.merge(left=despacho_directo_df,
                               right=intercompany_df,
                               left_on=['empresa_origen', 'empresa_destino'],
                               right_on=['empresa_origen', 'empresa_destino'],
                               how='left')

# Calcular el costo portuario por despacho directo
despacho_directo_df['costo_portuario_directo_total'] = despacho_directo_df['costo_portuario_directo_kg'] * despacho_directo_df['kilos_despachados_directo']
despacho_directo_df['costo_fletes_directo'] = despacho_directo_df['costo_flete_por_kg']*despacho_directo_df['kilos_despachados_directo'] + despacho_directo_df['costo_flete_por_camion'] * \
    despacho_directo_df['cantidad_camiones_directo']
despacho_directo_df['costo_intercompany_directo'] = despacho_directo_df['valor_cif']*despacho_directo_df['porcentaje_intercompany']*despacho_directo_df['kilos_despachados_directo']

# Totalizar despachos directos
despacho_directo_df = despacho_directo_df.groupby(['empresa_origen', 
                                                   'operador', 
                                                   'importacion', 
                                                   'ingrediente', 
                                                   'periodo'])[['cantidad_camiones_directo', 
                                                                'kilos_despachados_directo', 
                                                                'costo_portuario_directo_total', 
                                                                'costo_fletes_directo', 
                                                                'costo_intercompany_directo']].sum().reset_index().rename(columns={'empresa_origen':'empresa'})


# Despachos desde bodega
despacho_bodega_df = df_dict['despacho_bodega'].copy()
despacho_bodega_df.drop(columns=['tipo'])
despacho_bodega_df.rename(
    columns={'cantidad_camiones': 'cantidad_camiones_bodega'}, inplace=True)
despacho_bodega_df['kilos_despachados_bodega'] = 34000 * despacho_bodega_df['cantidad_camiones_bodega']

# Colocar fletes variables al despacho desde bodega
despacho_bodega_df = pd.merge(left=despacho_bodega_df,
                               right=fletes_variables_df,
                               left_on=['operador', 'ingrediente',
                                        'empresa_destino', 'planta'],
                               right_on=['operador', 'ingrediente',
                                         'empresa', 'planta'],
                               how='left').drop(columns=['empresa'])

# Colocar fletes fijos al despacho desde bodega
despacho_bodega_df = pd.merge(left=despacho_bodega_df,
                               right=fletes_fijos_df,
                               left_on=['operador', 'ingrediente',
                                        'empresa_destino', 'planta'],
                               right_on=['operador', 'ingrediente',
                                         'empresa', 'planta'],
                               how='left').drop(columns=['empresa'])

# Colocar valor Cif
despacho_bodega_df = pd.merge(left=despacho_bodega_df,
                               right=valor_cif_df,
                               left_on=['empresa_origen', 'operador',
                                        'importacion', 'ingrediente'],
                               right_on=['empresa', 'operador',
                                         'importacion', 'ingrediente'],
                               how='left').drop(columns=['empresa'])


# Colocar costo intercompany al despacho desde bodega
despacho_bodega_df = pd.merge(left=despacho_bodega_df,
                               right=intercompany_df,
                               left_on=['empresa_origen', 'empresa_destino'],
                               right_on=['empresa_origen', 'empresa_destino'],
                               how='left')


# Calcular el costo portuario por despacho desde bodega
despacho_bodega_df['costo_fletes_bodega'] = despacho_bodega_df['costo_flete_por_kg']*despacho_bodega_df['kilos_despachados_bodega'] + despacho_bodega_df['costo_flete_por_camion'] * \
    despacho_bodega_df['cantidad_camiones_bodega']
despacho_bodega_df['costo_intercompany_bodega'] = despacho_bodega_df['valor_cif']*despacho_bodega_df['porcentaje_intercompany']*despacho_bodega_df['kilos_despachados_bodega']

# Totalizar despachos desde bodega
despacho_bodega_df = despacho_bodega_df.groupby(['empresa_origen', 
                                                 'operador', 
                                                 'importacion', 
                                                 'ingrediente', 
                                                 'periodo'])[['cantidad_camiones_bodega', 
                                                              'kilos_despachados_bodega', 
                                                              'costo_fletes_bodega', 
                                                              'costo_intercompany_bodega']].sum().reset_index().rename(columns={'empresa_origen':'empresa'})

# Operacion de almacenamiento en puerto
bodegaje_puerto_df = df_dict['entradas_bodega_puerto'].copy()
bodegaje_puerto_df.drop(columns=['tipo'], inplace=True)
bodegaje_puerto_df.rename(columns={'kg':'bodegaje_puerto_kg'}, inplace=True)

# Unir inventarios en puerto con Costos de almacenamiento
fact_inventario_puerto = pd.merge(left=fact_inventario_puerto,
                                  right=costo_almacenamiento_df,
                                  left_on= ['empresa', 'operador', 'importacion', 'ingrediente', 'periodo'],
                                  right_on=['empresa', 'operador', 'importacion', 'ingrediente', 'periodo'],
                                  how='left')

# Unir inventarios en puerto con llegadas en barco
fact_inventario_puerto = pd.merge(left=fact_inventario_puerto,
                                  right=llegadas_puerto_df,
                                  left_on= ['empresa', 'operador', 'importacion', 'ingrediente', 'periodo'],
                                  right_on=['empresa', 'operador', 'importacion', 'ingrediente', 'periodo'],
                                  how='left')


# Colocarle a Fact los bodegajes en puerto
fact_inventario_puerto = pd.merge(left=fact_inventario_puerto,
                                  right=costo_op_bodegaje_df,
                                  left_on= ['operador', 'ingrediente'],
                                  right_on=['operador', 'ingrediente'],
                                  how='left')

# Colocarle a Fact los despachos directos
fact_inventario_puerto = pd.merge(left=fact_inventario_puerto,
                                  right=despacho_directo_df,
                                  left_on= ['empresa', 'operador', 'importacion', 'ingrediente', 'periodo'],
                                  right_on=['empresa', 'operador', 'importacion', 'ingrediente', 'periodo'],
                                  how='left')

# Colocarle a Fact los despachos desde bodega
fact_inventario_puerto = pd.merge(left=fact_inventario_puerto,
                                  right=despacho_bodega_df,
                                  left_on= ['empresa', 'operador', 'importacion', 'ingrediente', 'periodo'],
                                  right_on=['empresa', 'operador', 'importacion', 'ingrediente', 'periodo'],
                                  how='left')

# Colocarle a Fact los bodegajes en puerto
fact_inventario_puerto = pd.merge(left=fact_inventario_puerto,
                                  right=bodegaje_puerto_df,
                                  left_on= ['empresa', 'operador', 'importacion', 'ingrediente', 'periodo'],
                                  right_on=['empresa', 'operador', 'importacion', 'ingrediente', 'periodo'],
                                  how='left')

# Calcular Costos
fact_inventario_puerto['costo_portuario_bodegaje_total'] = fact_inventario_puerto['bodegaje_puerto_kg']*fact_inventario_puerto['costo_portuario_bodegaje_kg']

fact_inventario_puerto = fact_inventario_puerto.melt(id_vars=['empresa', 'operador', 'importacion', 'ingrediente', 'periodo'],
                                                     value_vars=['inventario_al_cierre_kg', 'costo_corte_por_kg',
                                                                 'descarga_barco_kg', 'costo_portuario_bodegaje_kg',
                                                                 'cantidad_camiones_directo', 'kilos_despachados_directo',
                                                                 'costo_portuario_directo_total', 'costo_fletes_directo',
                                                                 'costo_intercompany_directo', 'cantidad_camiones_bodega',
                                                                 'kilos_despachados_bodega', 'costo_fletes_bodega',
                                                                 'costo_intercompany_bodega', 'bodegaje_puerto_kg',
                                                                 'costo_portuario_bodegaje_total'],
                                                     value_name='kg', var_name='item')
fact_inventario_puerto['periodo'] = fact_inventario_puerto['periodo'].astype(int)
fact_inventario_puerto['kg'] = fact_inventario_puerto['kg'].apply(lambda x: round(x, 0))
fact_inventario_puerto = fact_inventario_puerto.pivot_table(values='kg',
                                                                    index=[
                                                                        'empresa', 'operador', 'importacion', 'ingrediente', 'item'],
                                                                    columns='periodo',
                                                                    aggfunc="sum").reset_index()

# Traer el calendario
calendario_df = df_dict['calendario'].copy()

# Armar un map para hacer un rename
date_map = {int(calendario_df.loc[r]['periodo']): calendario_df.loc[r]['value'].strftime('%Y-%m-%d') for r in calendario_df.index}
date_map[-1] = 'Previo'
fact_inventario_puerto.rename(columns=date_map, inplace=True)

In [ ]:
fact_inventario_puerto